# Walkable City Project

Problem:
I have received a job offer in Seattle and intend to move there, but I am not sure of the neighborhood that is best for me.  I currently live in the downtown area of Chicago and I would like to compare my neighborhood to a similar neighborhood in Seattle.  This would include walkability to grocery stores, proximity to restaurants and bars and access to public transportation.  Criteria will include number of grocery stores in neighborhood, the number of restaurants and bars in a neighborhood and the distance from the center of neighborhood to the closest public transit station, trains not buses. This system is for anyone moving from Chicago to Seattle.  It can be generalized for the user to select the venue types of most interest and what neighborhood or zip code they are moving from in Chicago.
Data:
The data needed will be the neighborhood names and zip codes for each of the cities (Chicago and Seattle) as well as the latitude and longitude data for the center of each of those neighborhoods.  Using that information, data will be downloaded from FourSquare to access grocery stores, restaurants (including coffee shops), bars and public transportation stops, but all the venue data will be stored for optional analysis.  A DataFrame of all of the neighborhoods will be constructed with each of those neighborhoods and a similarity score will be determined with Cosine Similarity.  After finding the neighborhood most similar, I will create clusters of venues based on lat/long data to find the location of the best place to live near a venue dense area and plot them on a map.


In [1]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import sklearn

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#Scraping data with beautiful soup
import urllib
import urllib.request
from bs4 import BeautifulSoup

## Importing and cleaning the Chicago Neighborhood data

In [5]:
df = pd.read_csv('https://data.cityofchicago.org/api/views/y6yq-dbs2/rows.csv?accessType=DOWNLOAD')

In [6]:
df['the_geom'] = df['the_geom'].str.strip('MULTIPOLYGON')
df['the_geom'] = df['the_geom'].str.replace('[()]','') 
df.head()

,the_geom,PRI_NEIGH,SEC_NEIGH,SHAPE_AREA,SHAPE_LEN
0,"-87.60670812560372 41.816813771373916, -87.60...",Grand Boulevard,BRONZEVILLE,4.849250e+07,28196.837157
1,"-87.62760697485348 41.8743709778537, -87.6275...",Printers Row,PRINTERS ROW,2.162138e+06,6864.247156
2,"-87.66706868914602 41.888851877695465, -87.66...",United Center,UNITED CENTER,3.252051e+07,23101.363745
3,"-87.65833494805533 41.92166144229183, -87.658...",Sheffield & DePaul,SHEFFIELD & DEPAUL,1.048259e+07,13227.049745
4,"-87.74059567509266 41.8878231689323, -87.7406...",Humboldt Park,HUMBOLDT PARK,1.250104e+08,46126.751351


In [7]:
lat = []
long = []
for i,x in enumerate(df['the_geom']):
    test = pd.Series(df.iloc[i,0].split(','))
    test2 = test.str.lstrip()
    test3 = pd.DataFrame(test2.str.split(' '),columns=['LatLong'])
    #print(test3['LatLong'].str[0])
    lat.append(test3['LatLong'].str[0].astype(float).mean())
    long.append(test3['LatLong'].str[1].astype(float).mean())


In [8]:
d = {'Neighborhood':df['PRI_NEIGH'],'Latitude':long,'Longitude':lat} #Coordinates were reversed
df_chi = pd.DataFrame(data=d)
df_chi.head()

,Neighborhood,Latitude,Longitude
0,Grand Boulevard,41.812351,-87.619113
1,Printers Row,41.871590,-87.629293
2,United Center,41.883024,-87.680273
3,Sheffield & DePaul,41.926751,-87.654054
4,Humboldt Park,41.902500,-87.714318


## Getting the venue data for Chicago

In [17]:
# The code was removed by Watson Studio for sharing.

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
Chi_Venues = getNearbyVenues(names=df_chi['Neighborhood'],
                                   latitudes=df_chi['Latitude'],
                                   longitudes=df_chi['Longitude']
                                  )

In [22]:
Chi_Venues.head()
Chi_Venues.shape

(6063, 7)

In [23]:
# Isolating my neightborhood.  This could be where a selection criteria can be put in so people can compare any Chicago neighborhood
df_hood = Chi_Venues[Chi_Venues['Neighborhood']=='West Loop']
df_hood.shape
df_hood.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2245,West Loop,41.878216,-87.647734,H-Mart,41.877857,-87.644980,Grocery Store
2246,West Loop,41.878216,-87.647734,Greek Islands,41.878922,-87.647328,Greek Restaurant
2247,West Loop,41.878216,-87.647734,"Artopolis Bakery, Cafe and Agora",41.877654,-87.647253,Greek Restaurant
2248,West Loop,41.878216,-87.647734,Ground Up,41.876710,-87.649558,Coffee Shop
2249,West Loop,41.878216,-87.647734,Mary Bartelme Park,41.879924,-87.650293,Park


In [24]:
# one hot encoding
hood_onehot = pd.get_dummies(df_hood[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hood_onehot['Neighborhood'] = df_hood['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hood_onehot.columns[-1]] + list(hood_onehot.columns[:-1])
hood_onehot = hood_onehot[fixed_columns]

hood_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,BBQ Joint,Bar,Big Box Store,Bookstore,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Café,Climbing Gym,Cocktail Bar,Coffee Shop,College Gym,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dog Run,Donut Shop,Fast Food Restaurant,Food Court,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Nightlife Spot,Park,Pet Store,Pizza Place,Portuguese Restaurant,Restaurant,Salad Place,Sandwich Place,Snack Place,Southern / Soul Food Restaurant,Sushi Restaurant,Taco Place,Wine Bar,Yoga Studio
2245,West Loop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2246,West Loop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2247,West Loop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2248,West Loop,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2249,West Loop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


#### This is where we could compare the names of places to major heading, but there are too many, and in accurate sub-headings, so I will compare my neighborhood
#### to those in Seattle without making modifiations. 

In [25]:
hood_grouped = hood_onehot.groupby('Neighborhood').mean().reset_index()
hood_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,BBQ Joint,Bar,Big Box Store,Bookstore,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Café,Climbing Gym,Cocktail Bar,Coffee Shop,College Gym,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dog Run,Donut Shop,Fast Food Restaurant,Food Court,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Nightlife Spot,Park,Pet Store,Pizza Place,Portuguese Restaurant,Restaurant,Salad Place,Sandwich Place,Snack Place,Southern / Soul Food Restaurant,Sushi Restaurant,Taco Place,Wine Bar,Yoga Studio
0,West Loop,0.01,0.01,0.02,0.03,0.01,0.01,0.01,0.02,0.02,0.01,0.03,0.01,0.03,0.07,0.01,0.01,0.02,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.05,0.03,0.02,0.02,0.01,0.03,0.02,0.05,0.02,0.01,0.01,0.01,0.02,0.03,0.02,0.01,0.02,0.01,0.04,0.01,0.03,0.01,0.02,0.01,0.01,0.02,0.01,0.02,0.01


In [26]:
# Remove the locations that are not as important to me
hood_grouped.drop(['Art Gallery','Cosmetics Shop','Dance Studio','Dog Run','Pet Store','Hotel','Yoga Studio'],axis=1,inplace=True)

## Importing Seattle Neighborhood Data

In [27]:
# The code was removed by Watson Studio for sharing.

  Neighborhood
0    Broadview
1  Bitter Lake
2  North Beach
3   Crown Hill
4    Greenwood (125, 1)


In [28]:
latitude = []
longitude = []
for x in df_sea['Neighborhood']:
    address = '{}, Seattle, WA',format(x)
#address = 'Broadview, Seattle, WA'
    #print(address)
    geolocator = Nominatim(user_agent="sea_explorer")
    location = geolocator.geocode(address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    

In [29]:
df_sea['lat'] = latitude
df_sea['long'] = longitude
df_sea.head()

,Neighborhood,lat,long
0,Broadview,47.722320,-122.360407
1,Bitter Lake,47.726236,-122.348764
2,North Beach,47.696210,-122.392362
3,Crown Hill,47.694715,-122.371459
4,Greenwood,47.690981,-122.354877


In [30]:
#Cleaning up some missing neighborhoods
df_sea.loc[df_sea['Neighborhood'] =='West Queen Anne','lat':'long'] = [47.634815, -122.370412] 
df_sea.loc[df_sea['Neighborhood'] =='North Queen Anne','lat':'long'] = [47.647287, -122.362249]
df_sea.loc[df_sea['Neighborhood'] =='East Queen Anne','lat':'long'] = [47.637105, -122.349565]
df_sea.loc[df_sea['Neighborhood'] =='Lower Queen Anne','lat':'long'] = [47.624644, -122.356730]
df_sea.loc[df_sea['Neighborhood'] =='Stevens','lat':'long'] = [47.626838, -122.302668]
df_sea.loc[df_sea['Neighborhood'] =='Central Waterfront','lat':'long'] = [47.611172, -122.344840]
df_sea.loc[df_sea['Neighborhood'] =='Miller Park','lat':'long'] = [47.621945, -122.306080]
df_sea.loc[df_sea['Neighborhood'] =='Pike/Pine','lat':'long'] = [47.614101, -122.320871]

In [31]:
geolocator = Nominatim(user_agent="sea_explorer")
location = geolocator.geocode(address)
sea_lat = location.latitude
sea_long = location.longitude

In [33]:
#Map for checking neighborhoods
map_sea = folium.Map(location=[sea_lat, sea_long], zoom_start=11)
# add markers to map
for lat, lng, neighborhood in zip(df_sea['lat'], df_sea['long'], df_sea['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sea)  
    
map_sea

## Getting Seattle Venue Data

In [34]:
Sea_Venues = getNearbyVenues(names=df_sea['Neighborhood'],
                                   latitudes=df_sea['lat'],
                                   longitudes=df_sea['long']
                                  )

In [35]:
print(Sea_Venues.shape)
print(Sea_Venues.head())

(7862, 7)
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Broadview               47.72232             -122.360407   
1    Broadview               47.72232             -122.360407   
2    Broadview               47.72232             -122.360407   
3    Broadview               47.72232             -122.360407   
4    Broadview               47.72232             -122.360407   

                  Venue  Venue Latitude  Venue Longitude     Venue Category  
0            Chada Thai       47.719663      -122.355114    Thai Restaurant  
1  Wicked Good Grinders       47.719657      -122.355966         Food Truck  
2   Broadview Tap House       47.719337      -122.356152           Beer Bar  
3                Redbox       47.719600      -122.355579        Video Store  
4              7-Eleven       47.719941      -122.355229  Convenience Store  


In [36]:
# one hot encoding
sea_onehot = pd.get_dummies(Sea_Venues[['Venue Category']], prefix="", prefix_sep="")
sea_onehot.drop('Neighborhood',axis=1,inplace=True)
# add neighborhood column back to dataframe
sea_onehot['Neighborhood'] = Sea_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sea_onehot.columns[-1]] + list(sea_onehot.columns[:-1])
sea_onehot = sea_onehot[fixed_columns]

sea_onehot.head()

,Neighborhood,ATM,Accessories Store,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cemetery,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Cafeteria,College Science Building,College Theater,Comedy Club,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Peking Duck Restaurant,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Post Office,Print Shop,Pub,Public Art,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Cente

In [37]:
sea_grouped = sea_onehot.groupby('Neighborhood').mean().reset_index()
sea_grouped.head()

,Neighborhood,ATM,Accessories Store,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cemetery,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Cafeteria,College Science Building,College Theater,Comedy Club,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Peking Duck Restaurant,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Post Office,Print Shop,Pub,Public Art,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Cente

## Matching Chicago Data to Seattle

In [42]:
chi_cols = hood_grouped.columns
sea_cols = sea_grouped.columns
cols = list(set(chi_cols) & set(sea_cols))

In [50]:
sea_matched = sea_grouped.loc[:,cols]
chi_matched = hood_grouped.loc[:,cols]
sea_matched.head()

,Café,Gym,Market,Coffee Shop,Italian Restaurant,New American Restaurant,Dessert Shop,Food Court,Korean Restaurant,Brewery,Bookstore,Climbing Gym,Breakfast Spot,Ice Cream Shop,Big Box Store,Cocktail Bar,Boxing Gym,Fast Food Restaurant,Grocery Store,Pizza Place,Japanese Restaurant,Donut Shop,Gym / Fitness Center,BBQ Joint,Sushi Restaurant,Diner,Bar,Southern / Soul Food Restaurant,Burger Joint,Salad Place,Greek Restaurant,Taco Place,Deli / Bodega,Martial Arts School,Mediterranean Restaurant,Mexican Restaurant,History Museum,Restaurant,Nightlife Spot,Sandwich Place,Snack Place,American Restaurant,Park,Neighborhood,Wine Bar
0,0.05,0.00,0.000000,0.090000,0.000000,0.01,0.02,0.00,0.04,0.02,0.02,0.0,0.01,0.010000,0.0,0.00,0.0,0.0,0.02,0.030000,0.01,0.000000,0.01,0.0,0.01,0.01,0.03,0.0,0.01,0.0,0.0,0.0,0.03,0.0,0.02,0.020000,0.000000,0.01,0.0,0.020000,0.0,0.00,0.020000,Adams,0.0
1,0.00,0.00,0.033333,0.066667,0.033333,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.033333,0.0,0.00,0.0,0.0,0.00,0.033333,0.00,0.033333,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.033333,0.033333,0.00,0.0,0.033333,0.0,0.00,0.033333,Alki Point,0.0
2,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.000000,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.250000,Arbor Heights,0.0
3,0.02,0.00,0.000000,0.000000,0.000000,0.00,0.02,0.01,0.01,0.01,0.01,0.0,0.00,0.000000,0.0,0.00,0.0,0.0,0.01,0.010000,0.01,0.010000,0.00,0.0,0.00,0.00,0.02,0.0,0.00,0.0,0.0,0.0,0.01,0.0,0.00,0.010000,0.010000,0.00,0.0,0.020000,0.0,0.02,0.010000,Atlantic,0.0
4,0.00,0.02,0.000000,0.080000,0.020000,0.01,0.02,0.00,0.01,0.00,0.01,0.0,0.01,0.050000,0.0,0.03,0.0,0.0,0.00,0.020000,0.00,0.010000,0.00,0.0,0.02,0.00,0.02,0.0,0.04,0.0,0.0,0.0,0.00,0.0,0.01,0.020000,0.000000,0.00,0.0,0.010000,0.0,0.00,0.020000,Ballard,0.0


In [51]:
chi_matched.head()

,Café,Gym,Market,Coffee Shop,Italian Restaurant,New American Restaurant,Dessert Shop,Food Court,Korean Restaurant,Brewery,Bookstore,Climbing Gym,Breakfast Spot,Ice Cream Shop,Big Box Store,Cocktail Bar,Boxing Gym,Fast Food Restaurant,Grocery Store,Pizza Place,Japanese Restaurant,Donut Shop,Gym / Fitness Center,BBQ Joint,Sushi Restaurant,Diner,Bar,Southern / Soul Food Restaurant,Burger Joint,Salad Place,Greek Restaurant,Taco Place,Deli / Bodega,Martial Arts School,Mediterranean Restaurant,Mexican Restaurant,History Museum,Restaurant,Nightlife Spot,Sandwich Place,Snack Place,American Restaurant,Park,Neighborhood,Wine Bar
0,0.03,0.02,0.01,0.07,0.05,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.01,0.03,0.01,0.01,0.03,0.04,0.02,0.01,0.02,0.02,0.02,0.01,0.03,0.01,0.01,0.01,0.05,0.01,0.03,0.01,0.02,0.03,0.01,0.03,0.01,0.02,0.01,0.01,0.02,West Loop,0.02


In [52]:
chi_input = np.array(chi_matched.drop('Neighborhood',axis=1))
x = chi_input.reshape(1,-1)
sea_array_in = sea_matched.drop('Neighborhood',axis = 1)
df_out = pd.DataFrame(data=sea_matched['Neighborhood'])
Similarity = []
for i,z in sea_array_in.iterrows(): 
    sea_input = np.array(z)
    y = sea_input.reshape(1,-1)
    Similarity.append(sklearn.metrics.pairwise.cosine_similarity(x, y))
#Similarity

### Finding the best neighborhood

In [53]:
for i,j in enumerate(Similarity):
    Similarity[i] = j[0][0]
df_out['Similarity'] = Similarity

In [54]:
df_out.iloc[df_out[['Similarity']].idxmax(),0]

69    Miller Park
Name: Neighborhood, dtype: object

# The Winner is Miller Park!